# Belief Adoption Model - Zilina with N=42971

This notebook demonstrates how to use the causalmp library to analyze counterfactual scenarios. We'll show how to:

- Set up simulation environments

- Generate experimental data

- Estimate counterfactual evolution under different intervention scenarios 

- Visualize results with informative plots

The notebook features parallel processing capabilities to speed up multiple experimental runs.

In [ ]:
# Import required libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import multiprocessing as mp
mp.set_start_method('fork', force=True)
from functools import partial
from tqdm.notebook import tqdm

# Import causalmp components
from causalmp import cmp_estimator, cmp_simulator

# 1. Environment Configuration
This environment models how beliefs spread through the Topolcany social network (18,246 nodes) under different treatment interventions, implementing Montanari and Saberi's (2010) cascade model. The experimental units are individual network members, with binary outcomes representing opinion adoption (1 for Opinion A, 0 for Opinion B) in each period. The treatment represents a campaign aimed at increasing Opinion A adoption, with effectiveness varying based on demographic factors from Pokec social network profiles. The opinion evolution follows a network-based coordination game where adoption probability depends on neighbor configurations and relative payoffs, creating complex patterns of direct and indirect influence that propagate through the moderately-sized community's social connections.

In [ ]:
# Define environment parameters for the Belief Adoption Model
environment = {
    'N': 42971,  # Number of nodes (Zilina network)
    'setting': "Belief Adoption Model",
    'design': [0, 0.1, 0.2, 0.5],  # Treatment probabilities at different stages
    'stage_time_blocks': [1, 3, 5, 7],  # Time periods for different stages
    'desired_design_1': [0, 0],  # Control scenario (no intervention)
    'desired_design_2': [0, 1],  # Treatment scenario (full intervention)
    'desired_stage_time_blocks': [1, 7],  # Time blocks for counterfactual scenarios
    'tau': 1  # Treatment effect coefficient 
}

data_path = None # Path to pre-generated data files (None means data will be freshly generated)

# 2. Estimation Parameters
Next, we configure the parameters for our counterfactual estimation models. These parameters control model complexity, regularization, and validation strategy.

In [ ]:
# Cross-validation parameters
n_validation_batch = 2  # Number of validation batches for model selection
time_blocks = [(0, 3), (3, 5), (5, 7)] # time blocks for validation in the format [(start1, end1), (start2, end2), ...]
N = environment['N']

# Detrending options
detrending_options = [True]  # Whether to detrend the data before estimation

# Detrending parameters (used only if detrending=True)
detrending_param_ranges = {
    'n_lags_Y_range': [1],  # Number of outcome lags
    'interaction_term_p_range': [None],  # Interaction term for population-level outcomes and treatment
    'interaction_term_u_range': [1],  # Interaction term for unit-level outcomes and treatment
    'n_batch_range': [1],  # Number of batches
    'batch_size_range': [N],  # Batch size
    'ridge_alpha_range': [1e-4]  # Regularization strength
}

# Main estimation parameters
main_param_ranges = {
    'n_lags_Y_range': [1],  # Number of outcome lags
    'n_lags_W_range': [1],  # Number of treatment lags
    'moment_order_p_Y_range': [1],  # Moment order for population-level outcomes
    'moment_order_p_W_range': [1],  # Moment order for population-level treatments,  
    'moment_order_u_Y_range': [1],  # Moment order for unit-level outcomes
    'moment_order_u_W_range': [1],  # Moment order for unit-level treatments, 
    'interaction_term_p_range': [None],  # Interaction term for population-level outcomes and treatment
    'interaction_term_u_range': [None, 1],  # Interaction term for unit-level outcomes and treatment
    'n_batch_range': [100, 500, 1000],  # Number of batches
    'batch_size_range': [int(0.05 * N), int(0.1 * N), int(0.2 * N), int(0.3 * N), int(0.5 * N)],  # Batch size
    'ridge_alpha_range': [1e-4, 1e-2, 1, 100]  # Regularization strength
}

# 3. Single Run Function
Here we define a function to run a single experiment. This function:

- Generates data using the simulator

- Runs the counterfactual cross-validation to choose the best model and configuration

- Estimates counterfactuals for the desired scenarios

- Computes treatment effects and organizes results

In [ ]:
def run_single_experiment(run_id, environment, main_param_ranges, 
                          detrending_options, detrending_param_ranges,
                          n_validation_batch, time_blocks,
                          visualize_ccv=False, data_path=None):
    """
    Run a single experiment with the Belief Adoption Model.
    
    Parameters:
    -----------
    run_id : int
        Identifier for this experimental run
    environment : dict
        Environment parameters
    main_param_ranges : dict
        Parameter ranges for the main estimator
    detrending_options : list
        Whether to use detrending
    detrending_param_ranges : dict
        Parameter ranges for detrending
    n_validation_batch : int
        Number of validation batches
    time_blocks : list of tuples
        Pre-defined time blocks for validation in the format [(start1, end1), (start2, end2), ...]
    visualize_ccv : bool
        Whether to visualize cross-validation results
    data_path : str or Path
        If provided, path to read the data instead of generating it
        
    Returns:
    --------
    tuple
        (observed_outcomes_df, CFEs_df, TTEs_df) containing results
    """
    print(f"\n--- Starting Run {run_id+1} ---")
    start_time = time.time()
    
    # Set random seed for reproducibility
    np.random.seed(run_id)
    
    # Step 1: Generate data using the simulator or load data
    if data_path is None:
        print("Generating data...")
        W, Y, desired_W_1, desired_Y_1, desired_W_2, desired_Y_2 = cmp_simulator(
            environment=environment,
            seed=run_id
        )
    else:
        print("Loading data...")
        W, Y, desired_W_1, desired_Y_1, desired_W_2, desired_Y_2 = load_data(
            environment=environment,
            data_path=data_path,
            run_id=run_id
        )
    print(f"Data shapes - W: {W.shape}, Y: {Y.shape}")
    
    # Step 2: Estimate counterfactuals
    print("Estimating counterfactuals...")
    predictions_1, predictions_2, best_config, best_model_terms = cmp_estimator(
        Y=Y,
        W=W,
        desired_W=desired_W_1,
        desired_W_2=desired_W_2,
        main_param_ranges=main_param_ranges,
        time_blocks=time_blocks,
        n_validation_batch=n_validation_batch,
        detrending_options=detrending_options,
        detrending_param_ranges=detrending_param_ranges,
        visualize_ccv=visualize_ccv,
        return_model_terms=True
    )
        
    # Step 4: Prepare results dataframes
    # Ground truth from simulation
    desired_CFE_1 = np.mean(desired_Y_1, axis=0)
    desired_CFE_2 = np.mean(desired_Y_2, axis=0)
    TTE_ground_truth = desired_CFE_2 - desired_CFE_1
    
    # Estimated treatment effects
    TTE_estimated = predictions_2 - predictions_1
    
    # Calculate Difference-in-Means and Horvitz-Thompson estimators
    from causalmp import dinm_estimate, ht_estimate, basic_cmp_estimate
    TTE_dim = dinm_estimate(Y, W)
    TTE_ht = ht_estimate(Y, W, environment["stage_time_blocks"], environment["design"])
    TTE_basic_cmp = basic_cmp_estimate(Y, W)
    
    # Create observed outcomes dataframe
    observed_mean = np.mean(Y, axis=0)
    observed_std = np.std(Y, axis=0)
    
    observed_outcomes_df = pd.DataFrame([
        *[{'Time': t, 'outcome': observed_mean[t], 'run': run_id, 'label': 'mean'} 
          for t in range(len(observed_mean))],
        *[{'Time': t, 'outcome': observed_std[t], 'run': run_id, 'label': 'stdev'} 
          for t in range(len(observed_std))]
    ])
    
    # Create CFEs dataframe
    CFEs_df = pd.DataFrame([
        *[{'Time': t, 'CFE': predictions_1[t], 'run': run_id, 
           'type': 'CFE(0)', 'label': 'Causal-MP'} 
          for t in range(len(predictions_1))],
        *[{'Time': t, 'CFE': predictions_2[t], 'run': run_id, 
           'type': 'CFE(1)', 'label': 'Causal-MP'} 
          for t in range(len(predictions_2))],
        *[{'Time': t, 'CFE': desired_CFE_1[t], 'run': run_id, 
           'type': 'CFE(0)', 'label': 'Ground Truth'} 
          for t in range(len(desired_CFE_1))],
        *[{'Time': t, 'CFE': desired_CFE_2[t], 'run': run_id, 
           'type': 'CFE(1)', 'label': 'Ground Truth'} 
          for t in range(len(desired_CFE_2))]
    ])
    
    # Create TTEs dataframe
    TTEs_df = pd.DataFrame([
        *[{'Time': t, 'TTE': TTE_ground_truth[t], 'run': run_id, 
           'label': 'GT'} 
          for t in range(len(TTE_ground_truth))],
        *[{'Time': t, 'TTE': TTE_estimated[t], 'run': run_id, 
           'label': 'CMP'} 
          for t in range(len(TTE_estimated))],
        *[{'Time': t, 'TTE': TTE_dim[t], 'run': run_id, 
           'label': 'DM'} 
          for t in range(len(TTE_dim))],
        *[{'Time': t, 'TTE': TTE_ht[t], 'run': run_id, 
           'label': 'HT'} 
          for t in range(len(TTE_ht))],
        *[{'Time': t, 'TTE': TTE_basic_cmp[t], 'run': run_id, 
           'label': 'bCMP'} 
          for t in range(len(TTE_basic_cmp))]
    ])
    
    elapsed_time = time.time() - start_time
    print(f"Run {run_id+1} completed in {elapsed_time:.2f} seconds")
    
    return observed_outcomes_df, CFEs_df, TTEs_df, best_config, best_model_terms

def load_data(environment, data_path, run_id):
        """Load data from files."""
        from pathlib import Path
        # Extract parameters
        N = environment["N"]
        T = environment["stage_time_blocks"][-1]
        
        file_paths = {
            'Y': Path(data_path) / f'seed{run_id}/experiment_n{N}_t{T}_seed{run_id}_panel_data.csv',
            'Y_1': Path(data_path) / f'seed{run_id}/treatment_n{N}_t{T}_seed{run_id}_panel_data.csv',
            'Y_0': Path(data_path) / f'seed{run_id}/control_n{N}_t{T}_seed{run_id}_panel_data.csv',
            'W': Path(data_path) / f'seed{run_id}/experiment_n{N}_t{T}_seed{run_id}_treatment_data.csv',
            'W_1': Path(data_path) / f'seed{run_id}/treatment_n{N}_t{T}_seed{run_id}_treatment_data.csv',
            'W_0': Path(data_path) / f'seed{run_id}/control_n{N}_t{T}_seed{run_id}_treatment_data.csv'
        }
        
        # Check for missing files
        missing = [f for f in file_paths.values() if not f.exists()]
        if missing:
            raise FileNotFoundError(f"Missing files: {missing}")
        
        # Read all files
        data = {k: pd.read_csv(v).iloc[:, 0:].to_numpy() for k, v in file_paths.items()}
        
        return (data['W'], data['Y'], data['W_0'], data['Y_0'], data['W_1'], data['Y_1'])

# 4. Multiple Runs Function
This function orchestrates multiple experiment runs, with support for parallel processing to speed up computation.

In [ ]:
def run_multiple_experiments(n_runs, n_processes=None, data_path=None, 
                             return_model_terms=True, **experiment_params):
    """
    Run multiple experimental runs, either sequentially or in parallel.
    
    Parameters:
    -----------
    n_runs : int
        Number of experimental runs
    n_processes : int or None
        Number of parallel processes (None or 1 for sequential)
    **experiment_params : dict
        Parameters to pass to each experiment
        
    Returns:
    --------
    tuple
        (all_observed_outcomes, all_CFEs, all_TTEs) as DataFrames
    """
    start_time = time.time()
    
    # Initialize combined DataFrames
    all_observed_outcomes = pd.DataFrame()
    all_CFEs = pd.DataFrame()
    all_TTEs = pd.DataFrame()
    
    # Sequential execution
    if n_processes is None or n_processes == 1:
        print(f"\nRunning {n_runs} experiments sequentially...")
        for run_id in tqdm(range(n_runs)):
            observed_df, cfes_df, ttes_df, best_config, best_model_terms = run_single_experiment(
                data_path=data_path, run_id=run_id, **experiment_params
            )
            
            # Display best configuration
            if return_model_terms:
                from causalmp import ResultVisualizer
                visualizer = ResultVisualizer()
                visualizer.display_best_configuration(best_config, best_model_terms)
            
            # Concatenate results
            all_observed_outcomes = pd.concat([all_observed_outcomes, observed_df], ignore_index=True)
            all_CFEs = pd.concat([all_CFEs, cfes_df], ignore_index=True)
            all_TTEs = pd.concat([all_TTEs, ttes_df], ignore_index=True)
            
    # Parallel execution
    else:
        print(f"\nRunning {n_runs} experiments in parallel with {n_processes} processes...")
        n_processes = min(n_processes, mp.cpu_count())
        
        # Add collection for best configs and model terms
        best_configs_dict = {}
        best_model_terms_dict = {}
        
        # Create partial function with fixed experiment parameters
        partial_run = partial(run_single_experiment, data_path=data_path, **experiment_params)
        
        # Run parallel processing
        with mp.Pool(processes=n_processes) as pool:
            results = list(tqdm(pool.imap(partial_run, range(n_runs)), total=n_runs))
            
            # Process results
            for i, (observed_df, cfes_df, ttes_df, best_config, best_model_terms) in enumerate(results):
                all_observed_outcomes = pd.concat([all_observed_outcomes, observed_df], ignore_index=True)
                all_CFEs = pd.concat([all_CFEs, cfes_df], ignore_index=True)
                all_TTEs = pd.concat([all_TTEs, ttes_df], ignore_index=True)
                best_configs_dict[i] = best_config
                best_model_terms_dict[i] = best_model_terms
            
        # Display best configuration
        if return_model_terms:
            from causalmp import ResultVisualizer
            visualizer = ResultVisualizer()
            for i in range(n_runs):
                visualizer.display_best_configuration(best_configs_dict[i],
                                                      best_model_terms_dict[i])
    
    total_time = time.time() - start_time
    print(f"\nAll experiments completed in {total_time:.2f} seconds")
    print(f"Average time per run: {total_time/n_runs:.2f} seconds")
    
    return all_observed_outcomes, all_CFEs, all_TTEs

# 5. Run the Experiments
Now we'll execute the experiments with our defined parameters.

In [ ]:
# User-configurable parameters
n_runs = 100  # Number of experimental runs
n_processes = 100  # Number of parallel processes (None or 1 for sequential execution)

# Run the experiments
all_observed_outcomes, all_CFEs, all_TTEs = run_multiple_experiments(
    n_runs=n_runs,
    n_processes=n_processes,
    return_model_terms=True,
    environment=environment,
    main_param_ranges=main_param_ranges,
    detrending_options=detrending_options,
    detrending_param_ranges=detrending_param_ranges,
    n_validation_batch=n_validation_batch,
    time_blocks=time_blocks,
    data_path=data_path,
    visualize_ccv=False
)

# Display summary statistics
print("\nSummary Statistics:")
print(f"Number of runs: {n_runs}")
print(f"Unique runs in data: {len(all_observed_outcomes['run'].unique())}")
print(f"Time periods: {all_observed_outcomes['Time'].max() + 1}")

# 6. Visualize and Interpret Results
Finally, we'll visualize the results and interpret our findings.

In [ ]:
# Import the ResultVisualizer from causalmp
from causalmp import ResultVisualizer

# Set plot styling
sns.set_theme(style="whitegrid")

# Create a visualizer instance
visualizer = ResultVisualizer()

# Visualize the results using the built-in visualizer
visualizer.plot_results(
    Observed_outcomes=all_observed_outcomes,
    CFEs=all_CFEs,
    TTEs=all_TTEs,
    # Optional parameters:
    filename="belief_adoption_Z.pdf",  # Uncomment to save the plot to a file
    # layout="1x4",  # Options: "1x4" or "2x2"
    y_lim=(0.0, 0.031),  # Optional y-axis limits for TTE plot
    # x_ticks=list(range(0, environment['stage_time_blocks'][-1] + 1, 5))  # Optional custom x-ticks
    n_periods_for_tte=2
)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a figure with a specific size
plt.figure(figsize=(12, 8))

# Your original plot code
sns.lineplot(data=all_TTEs,
             x='Time',
             y='TTE',
             errorbar=('pi', 95),
             hue='label',
             style='label')

plt.ylim(bottom=0, top=40)  

# Apply layout adjustments
plt.tight_layout()
plt.show()